# Homework 2

## Reading
* Descriptor: https://realpython.com/python-descriptors/#how-attributes-are-accessed-with-the-lookup-chain
* Data model: https://docs.python.org/3/reference/datamodel.html (contents related to what we taught)
* Python MRO: https://www.python.org/download/releases/2.3/mro/

## Merge sorted stream
Write a generator function that takes two sorted streams (generators), and return a generator that can produce a merged stream in sorted order.

Bonus point: can you make it generic such that it can merge any number of streams?


```python
def merge_sorted_stream(stream1, stream2):
    yield ...

stream1 = range(0, 10, 2)
stream2 = range(1, 10, 2)

for x in merge_sorted_stream(stream1, stream2):
    print(x)

0 
1
2
3
4
5
6
7
8
9
```

In [21]:
def merge_sorted_stream(stream1, stream2):
    itr1 = iter(stream1)
    itr2 = iter(stream2)
    
    itr1_empty = False
    itr2_empty = False
    
    v1 = None
    v2 = None
    
    try:
        v1 = next(itr1)
    except StopIteration:
        itr1_empty = True
        
    try: 
        v2 = next(itr2)
    except StopIteration:
        itr2_empty = True
    
    while not itr1_empty and not itr2_empty:
        if v1 <= v2:
            temp = v1
            
            try:
                v1 = next(itr1)
            except StopIteration:
                itr1_empty = True
                
            yield temp
            
        else:
            temp = v2
            
            try:
                v2 = next(itr2)
            except StopIteration:
                itr2_empty = True
                
            yield temp
            
    while not itr2_empty:
        temp = v2
            
        try:
            v2 = next(itr2)
        except StopIteration:
            itr2_empty = True

        yield temp
        
    while not itr1_empty:
        temp = v1
            
        try:
            v1 = next(itr1)
        except StopIteration:
            itr1_empty = True

        yield temp
        
    
                
        
    

In [24]:
stream1 = range(0, 10, 2)
stream2 = range(1, 10, 2)
for x in merge_sorted_stream(stream1, stream2):
    print(x)

1
3
5
7
9


In [25]:
stream1 = range(0, 10, 1)
stream2 = range(1, 10, 2)
for x in merge_sorted_stream(stream1, stream2):
    print(x)

0
1
1
2
3
3
4
5
5
6
7
7
8
9
9


In [26]:
stream1 = range(0, 0)
stream2 = range(1, 10, 2)
for x in merge_sorted_stream(stream1, stream2):
    print(x)

1
3
5
7
9


## Bonus Question

In [54]:
from heapq import *
def merge_generic_number_of_streams(*args):
    k = len(args)
    itr_k = [iter(val) for val in args]
    itr_empty = [False] * k
    value = [None] * k
    
    for i in range(k):
        try:
            value[i] = next(itr_k[i])
        except StopIteration:
            itr_empty[i] = True
            
    heap = [(value[i], i, itr_k[i]) for i in range(k) if value[i] is not None]
    heapify(heap)
    
    while heap:

        val, i, itr = heappop(heap)
        yield val
        try:
            value[i] = next(itr)
        except StopIteration:
            itr_empty[i] = True
            value[i] = None
        else:
            heappush(heap, (value[i], i, itr))
               
    

In [56]:
stream1 = range(0, 15, 3)
stream2 = range(1, 15, 3)
stream3 = range(2, 15, 3)
for x in merge_generic_number_of_streams(stream1, stream2, stream3):
    print(x)

0
1
2
3
4
5
6
7
8
9
10
11
12
13
14


## Tree traversal

Unlike linear data structures (Array, Linked List, Queues, Stacks, etc) which have only one logical way to traverse them, trees can be traversed in different ways. Following are the generally used ways for traversing trees.

```
      1
    /  \ 
   2    3
  / \
 4   5
```

Depth First Traversals: 
  * (a) Inorder (Left, Root, Right) : 4 2 5 1 3
  * (b) Preorder (Root, Left, Right) : 1 2 4 5 3
  * (c) Postorder (Left, Right, Root) : 4 5 2 3 1

Define a Tree class with a method that can walk through the tree in different orders. Hint: use generator will make your life a lot easier.

```python
class TreeNode:
    
    ...

    def in_order(self):
        pass
        
    def pre_order(self):
        pass
        
    def post_order(self):
        pass

    
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)

>>> print(' -> '.join(item for item in root.in_order()))
4 -> 2 -> 5 -> 1 -> 3    


```
   

In [62]:
class TreeNode:
    def __init__(self, val):
        self.val = val
        self.left = None
        self.right = None

    def in_order(self):
        p = self
        s = []
        
        while p or s:
            while p:
                s.append(p)
                p = p.left
                
            if s:
                p = s[-1]
                yield p.val
                s.pop()
                p = p.right

    def pre_order(self):
        p = self
        s = []
        
        while p or s:
            while p:
                s.append(p)
                yield p.val
                p = p.left
                
            if s:
                p = s[-1]
                s.pop()
                p = p.right
        
    def post_order(self):
        p = self
        s = []
        pre = None
        
        s.append(p)
        
        while s:
            cur = s[-1]
            if (not cur.left and not cur.right) or (pre and (pre == cur.left or pre == cur.right)):
                yield cur.val
                s.pop()
                pre = cur
                
            else:
                if cur.right:
                    s.append(cur.right)
                if cur.left:
                    s.append(cur.left)
        

In [63]:
root = TreeNode(1)
root.left = TreeNode(2)
root.right = TreeNode(3)
root.left.left = TreeNode(4)
root.left.right = TreeNode(5)
print(' -> '.join(str(item) for item in root.pre_order()))
print(' -> '.join(str(item) for item in root.in_order()))
print(' -> '.join(str(item) for item in root.post_order()))

1 -> 2 -> 4 -> 5 -> 3
4 -> 2 -> 5 -> 1 -> 3
4 -> 5 -> 2 -> 3 -> 1


## Implement a timer
Implement a timer that can print the execution time of your code. Try to implement it both as a decorator and as a context manager to compare the implementations. Can you implement it using one single class? 

Example:
```python
import time


with timer() as timer:
    time.sleep(3)

# Total execution time: 3.000123 seconds (a made up number)


@timer
def sleep(secs):
    time.sleep(secs)
    
sleep(3)
# Total execution time: 3.000123 seconds (a made up number)
```

Below is the code snippet to measure the time:
```python    
import time
start_time = time.time()
main()
print(f"--- {time.time() - start_time} seconds ---"
```

In [141]:
import time
import functools
class timer:
    
    def __init__(self, what=None):
        self.what = what #For decorator, the function is passed in here
    
    def __call__(self, *sec): #the parameteres are passed in here
        '''
            Decorator Method
        '''
        start_time = time.time()
        self.what(*sec)
        print(f"--- {time.time() - start_time} seconds ---")
        
    
    '''
        Context Manager
    '''
    def __enter__(self):
        self.start_time = time.time()
        
    def __exit__(self, exc_type, exc_val, exc_tb):
        print(f"--- {time.time() - self.start_time} seconds ---")
            

In [142]:
#Decorator
@timer
def sleep(secs):
    time.sleep(secs)
    
sleep(1)

--- 1.0007085800170898 seconds ---


In [140]:
#Context Manager
with timer() as timer:
    time.sleep(1)

--- 1.0005419254302979 seconds ---


In [144]:
@timer
def running(a, b, c):
    return [i for i in range(a, b, c)]

running(1, 100000, 2)

--- 0.003991127014160156 seconds ---


In [145]:
with timer() as timer:
    for i in range(1, 100000, 2):
        continue

--- 0.001994609832763672 seconds ---
